Column Transfromer : it is a new class of sklearn from which we can get the numpy array using a single line of code.

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv('covid_toy.csv')

In [5]:
df.head(10)

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No
5,84,Female,NaN,Mild,Bangalore,Yes
6,14,Male,101.0,Strong,Bangalore,No
7,20,Female,NaN,Strong,Mumbai,Yes
8,19,Female,100.0,Strong,Bangalore,No
9,64,Female,101.0,Mild,Delhi,No


In [6]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [7]:
df['cough'].value_counts()

Mild      62
Strong    38
Name: cough, dtype: int64

In [8]:
df['city'].value_counts()

Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: city, dtype: int64

In [9]:
df['gender'].value_counts()

Female    59
Male      41
Name: gender, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split

In [22]:
X = df.iloc[:,0:5]
Y = df.iloc[:,-1:]

In [24]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2)

# Without using Sklearn:

In [28]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [29]:
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])
X_test_fever = si.fit_transform(X_test[['fever']])

In [30]:
X_train_fever.shape

(80, 1)

In [32]:
oe = OrdinalEncoder(categories =[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])
X_test_cough = oe.fit_transform(X_test[['cough']])

In [33]:
X_train_cough.shape

(80, 1)

In [36]:
ohe = OneHotEncoder(drop = 'first', sparse_output = False)

In [37]:
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

In [38]:
X_test_gender_city.shape

(20, 4)

In [43]:
X_train_age = X_train.drop(columns = ['gender','fever','cough','city']).values

In [44]:
X_test_age = X_test.drop(columns = ['gender','fever','cough','city']).values

In [45]:
X_train_age.shape

(80, 1)

In [48]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

# Using Sklearn column Transformer

In [49]:
from sklearn.compose import ColumnTransformer

In [54]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [58]:
transformer.fit_transform(X_train)

array([[100.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  47.        ],
       [104.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  18.        ],
       [101.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   8.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  81.        ],
       [ 99.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  59.        ],
       [102.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,   5.        ],
       [103.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  69.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  71.        ],
       [100.        ,   0.        ,   0.        ,   0.        ,
          1.    

In [57]:
transformer.fit_transform(X_test)

array([[104.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  17.        ],
       [100.23529412,   1.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  71.        ],
       [ 99.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  14.        ],
       [102.        ,   1.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  20.        ],
       [100.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  27.        ],
       [100.23529412,   0.        ,   1.        ,   0.        ,
          0.        ,   1.        ,  23.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ,   5.        ],
       [100.23529412,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  82.        ],
       [ 98.        ,   1.        ,   1.        ,   0.        ,
          0.    